In [74]:
import csv
import pandas as pd
import igraph as ig
from igraph import *

In [75]:
data_file = '/home/dipp/Github/Master-Thesis-dipp/Network_On_Global_Scale/Version 2 Latest/Data/Location_with_only_country_code_V2_Latest.csv'

In [76]:
country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            if(row['i']!=row['j']):
                souce_loc = row['i'].replace("'","")
                destination_loc = row['j'].replace("'","")
                #country_data[count] = [row['i'], row['j'], float(row['count'])]
                if(float(row['count']) > 999):
                    country_data[count] = [souce_loc, destination_loc, float(row['count'])]
                    count+=1

In [77]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['i','j','count'])

In [78]:
df

,i,j,count
0,US,RU,31451.0
1,GB,MY,29532.0
2,KR,US,64503.0
3,BE,US,21710.0
4,GB,MX,52770.0
...,...,...,...
636,CN,CU,1340.0
637,AU,ES,1150.0
638,PL,IN,1075.0
639,AU,CO,1032.0


In [79]:
g = ig.Graph.TupleList(df.itertuples(index=False), directed=True, edge_attrs="count")

In [80]:
deg = g.strength(mode='all', loops='True', weights='count')
degree_data_df = pd.DataFrame({attr: g.vs[attr] for attr in g.vertex_attributes()})
##This is the Node
degree_data_df['Degree'] = deg

In [81]:
degree_data_df

,name,Degree
0,US,9396775.0
1,RU,43681.0
2,GB,3464541.0
3,MY,189675.0
4,KR,118539.0
...,...,...
111,MU,2260.0
112,JE,2178.0
113,CW,1568.0
114,TT,1139.0


In [82]:
degree_dict = {}
for index, row in degree_data_df.iterrows():
    degree_dict[row['name']] = row['Degree']

In [83]:
vi = []
vj = []
for index, row in df.iterrows():
    ai = row['i']
    bi = row['j']

    vi.append(degree_dict.get(ai))
    vj.append(degree_dict.get(bi))

In [84]:
df['vi'] = vi
df['vj'] = vj

In [85]:
df

,i,j,count,vi,vj
0,US,RU,31451.0,9396775.0,43681.0
1,GB,MY,29532.0,3464541.0,189675.0
2,KR,US,64503.0,118539.0,9396775.0
3,BE,US,21710.0,120643.0,9396775.0
4,GB,MX,52770.0,3464541.0,351010.0
...,...,...,...,...,...
636,CN,CU,1340.0,123815.0,69618.0
637,AU,ES,1150.0,706182.0,195550.0
638,PL,IN,1075.0,61683.0,1380603.0
639,AU,CO,1032.0,706182.0,252054.0


In [86]:
percentage_of_incoming = []
for index, row in df.iterrows():
    nij = row['count']
    njk = row['vj']

    per_incoming = (nij / njk) * 100

    percentage_of_incoming.append(float(per_incoming))


df['Percentage of Incoming'] = percentage_of_incoming

In [87]:
cij = []
for index, row in df.iterrows():
    nij = row['count']
    nik = row['vi']
    njk = row['vj']

    temp_cij = nij / (nik + njk)

    cij.append(float(temp_cij))

In [88]:
df['cij'] = cij

In [89]:
df


,i,j,count,vi,vj,Percentage of Incoming,cij
0,US,RU,31451.0,9396775.0,43681.0,72.001557,0.003332
1,GB,MY,29532.0,3464541.0,189675.0,15.569790,0.008082
2,KR,US,64503.0,118539.0,9396775.0,0.686438,0.006779
3,BE,US,21710.0,120643.0,9396775.0,0.231037,0.002281
4,GB,MX,52770.0,3464541.0,351010.0,15.033760,0.013830
...,...,...,...,...,...,...,...
636,CN,CU,1340.0,123815.0,69618.0,1.924790,0.006927
637,AU,ES,1150.0,706182.0,195550.0,0.588085,0.001275
638,PL,IN,1075.0,61683.0,1380603.0,0.077865,0.000745
639,AU,CO,1032.0,706182.0,252054.0,0.409436,0.001077


In [91]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Network_On_Global_Scale/Version 2 Latest/Result/global_network_test_2.csv', index=False)

In [90]:
new_df = df.sort_values(by='Percentage of Incoming', ascending=False)
new_df

,i,j,count,vi,vj,Percentage of Incoming,cij
90,US,EE,1165.0,9396775.0,1165.0,100.000000,0.000124
333,US,MV,1504.0,9396775.0,1504.0,100.000000,0.000160
556,US,CR,2014.0,9396775.0,2014.0,100.000000,0.000214
460,US,BH,1103.0,9396775.0,1103.0,100.000000,0.000117
135,US,SD,2417.0,9396775.0,2417.0,100.000000,0.000257
...,...,...,...,...,...,...,...
618,RS,US,1180.0,10190.0,9396775.0,0.012557,0.000125
266,KP,US,1124.0,1124.0,9396775.0,0.011962,0.000120
143,KW,US,1071.0,22880.0,9396775.0,0.011398,0.000114
492,SI,US,1056.0,7033.0,9396775.0,0.011238,0.000112


## TEST 2

In [92]:
import csv
import pandas as pd
import networkx as nx

In [93]:
data_file = '/home/dipp/Github/Master-Thesis-dipp/Network_On_Global_Scale/Version 2 Latest/Data/Location_with_only_country_code_V2_Latest.csv'

In [94]:
country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            if(row['i']!=row['j']):
                souce_loc = row['i'].replace("'","")
                destination_loc = row['j'].replace("'","")
                #country_data[count] = [row['i'], row['j'], float(row['count'])]
                if(float(row['count']) > 999):
                    country_data[count] = [souce_loc, destination_loc, float(row['count'])]
                    count+=1

In [95]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['i','j','count'])
df

,i,j,count
0,US,RU,31451.0
1,GB,MY,29532.0
2,KR,US,64503.0
3,BE,US,21710.0
4,GB,MX,52770.0
...,...,...,...
636,CN,CU,1340.0
637,AU,ES,1150.0
638,PL,IN,1075.0
639,AU,CO,1032.0


In [99]:
G = nx.from_pandas_edgelist(df, 'i', 'j', ['count'])

In [113]:
G.degree(weight='count')['US']

3941724.0

In [110]:
country = []
deg = G.degree(weight='count')
degree_data_df['Degree'] = deg

In [118]:
degree_data_df

,name,Degree
0,US,"(US, 3941724.0)"
1,RU,"(RU, 9052.0)"
2,GB,"(GB, 1613414.0)"
3,MY,"(MY, 69004.0)"
4,KR,"(KR, 38438.0)"
...,...,...
111,MU,"(MU, 2260.0)"
112,JE,"(JE, 2178.0)"
113,CW,"(CW, 1568.0)"
114,TT,"(TT, 1139.0)"


In [115]:
for i,j in deg:

SyntaxError: incomplete input (986015291.py, line 1)